In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64
training_dataloader = DataLoader(training_data, batch_size=batch_size)
testing_dataloader = DataLoader(testing_data, batch_size=batch_size)

In [4]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

class ClothingClassNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

In [5]:
model = ClothingClassNeuralNetwork().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [6]:
def train_model(model, dataloader, loss_function, optimizer):
    dataset_size = len(dataloader.dataset)
    model.train()

    for index, (input_tensor, target) in enumerate(dataloader):
        input_tensor, target = input_tensor.to(device), target.to(device)

        prediction = model(input_tensor)
        loss = loss_function(prediction, target)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if index % 100 == 0:
            loss, current = loss.item(), (index + 1) * len(input_tensor)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

In [7]:
def test_model(model, dataloader, loss_function):
    size = len(dataloader.dataset)
    batch_length = len(dataloader)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for input_tensor, target in dataloader:
            input_tensor, target = input_tensor.to(device), target.to(device)
            prediction = model(input_tensor)
            loss += loss_function(prediction, target).item()
            correct += (prediction.argmax(1) == target).type(torch.float).sum().item()
    loss /= batch_length
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    

In [12]:
epochs = 5
for epoch_iteration in range(epochs):
    train_model(model, training_dataloader, loss_function, optimizer)
    test_model(model, testing_dataloader, loss_function)
print("Complete.")

loss: 0.359212  [   64/60000]
loss: 0.487420  [ 6464/60000]
loss: 0.307069  [12864/60000]
loss: 0.542860  [19264/60000]
loss: 0.461149  [25664/60000]
loss: 0.460182  [32064/60000]
loss: 0.489269  [38464/60000]
loss: 0.653409  [44864/60000]
loss: 0.609227  [51264/60000]
loss: 0.434209  [57664/60000]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.487960 

loss: 0.355776  [   64/60000]
loss: 0.485412  [ 6464/60000]
loss: 0.304955  [12864/60000]
loss: 0.540105  [19264/60000]
loss: 0.458024  [25664/60000]
loss: 0.457664  [32064/60000]
loss: 0.487271  [38464/60000]
loss: 0.651906  [44864/60000]
loss: 0.607208  [51264/60000]
loss: 0.431621  [57664/60000]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.486209 

loss: 0.352460  [   64/60000]
loss: 0.483493  [ 6464/60000]
loss: 0.302888  [12864/60000]
loss: 0.537444  [19264/60000]
loss: 0.455025  [25664/60000]
loss: 0.455159  [32064/60000]
loss: 0.485332  [38464/60000]
loss: 0.650301  [44864/60000]
loss: 0.605272  [51264/60000]
loss: 0.429190  [57664/600